# Covid-19 vs Pneumo Xray Detection - Testing Application

<img src="../images/portada.png"/>
Developed by M.Rovai @ May,14 2020<br>

Based on the word of [Adrian Rosebrock](https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/) and [Nell Trevor](https://www.kaggle.com/mauddib/covid-19-x-ray-deep-learning-api-endpoint-code)


## Disclaimer

The deployment of an automatic COVID-19 detection is for educational purposes only. It is not meant to be a reliable, highly accurate COVID-19 diagnosis system, nor has it been professionally or academically vetted.

## Introduction

**Inspiration**

The inspiration of this project, was to understand and create a didactic proof of concept of the work "[XRayCovid-19](http://tools.atislabs.com.br/covid)" developed by UFRRJ (Universidade Federal Rural do Rio de Janiero). **XRayCovid-19** is an ongoing project that uses Artificial Intelligence to assist the health system in the COVID-19 diagnostic process. It is characterized by easy use; efficiency in response time and effectiveness in the result.

**Why X-rays?**

There have been promising efforts to apply machine learning to aid in the diagnosis of COVID-19 based on CT scans. Despite the success of these methods, the fact remains that COVID-19 is an infection that is likely to be experienced by communities of all sizes. X-rays are inexpensive and quick to perform; therefore, they are more accessible to healthcare providers working in smaller and/or remote regions. 

**Thanks**

This work was developed using TensorFlow and Keras, based on the great tutorial published by [Dr. Adrian Rosebrock](https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/). Also, I would like to thanks Nell Trevor that, also based on Dr. Rosebrock's work, provided an endpoint idea, where the resultant model could be tested: [Covid-19 predictor API](http://coviddetector.pythonanywhere.com)

## Papers

- [[1] 2020 Chowdhury et al - Can AI help in screening Viral and COVID-19 pneumonia?](https://arxiv.org/pdf/2003.13145.pdf)
- [[2] 2020 Hall et all - Finding COVID-19 from Chest X-rays using Deep Learning on a Small Dataset](https://arxiv.org/pdf/2004.02060.pdf)
- [[3] 2020 COVID-19 Screening on Chest X-ray Images Using Deep Learning based Anomaly Detection](https://arxiv.org/pdf/2003.12338.pdf)

## Import Libraries and Setup

In [16]:
import cv2
from tensorflow.keras.models import load_model

## Support Functions

In [10]:
def test_rx_image_for_Covid19_2(model, imagePath):
    img = cv2.imread(imagePath)
    img_out = img
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)

    img = np.array(img) / 255.0

    pred = model.predict(img)
    pred_neg = round(pred[0][1]*100)
    pred_pos = round(pred[0][0]*100)
    
    if np.argmax(pred, axis=1)[0] == 1:
        prediction = 'NEGATIVE'
        prob = pred_neg
    else:
        prediction = 'POSITIVE'
        prob = pred_pos

    cv2.imwrite('../Image_Prediction/Image_Prediction.png', img_out)
    return prediction, prob

## General Technical overview

**What is the difference between an AP and a PA view of an X-ray?**
in simple words, during the procedure of taking x-ray radiograph when x-ray passes from posterior of body to anterior, it is called PA view. While in AP view the direction is opposite.

Now let's understand the importance of doing this. According to concepts of physics the less the distance between the object and screen more clearer shadow is formed. In this case x-ray is the light source and we are the object. So, in AP view posterior aspect gives better shadow, while in PA view anterior aspect gives better shadow.

Mostly the x-ray is taken in AP view for any body part. An important exception here is the Chest. In Chest, we prefer the **PA view** over AP view. But if the patient is very ill and not able to maintain his position then AP view can be done for chest.

<img src="../images/ap_pa_chest.jpg"/>

From exam point of view, any x-ray you see in general is AP, or L (Lateral) and any chest x-ray is PA view mostly.

## Loading Model

In [3]:
covid_pneumo_model = load_model('../model/covid_pneumo_model.h5')

# Show the model architecture
covid_pneumo_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Testing model with Real images

### Test on Covid images (From March/2020)

In [7]:
!ls ../dataset_validation/covid_validation/

2966893D-5DDF-4B68-9E2B-4979D5956C8E.jpeg
31BA3780-2323-493F-8AED-62081B9C383B.jpeg
6C94A287-C059-46A0-8600-AFB95F4727B7.jpeg
7C69C012-7479-493F-8722-ABC29C60A2DD.jpeg
B2D20576-00B7-4519-A415-72DE29C90C34.jpeg
E63574A7-4188-4C8D-8D17-9D67A18A1AFA.jpeg
F2DE909F-E19C-4900-92F5-8F435B031AC6.jpeg
F4341CE7-73C9-45C6-99C8-8567A5484B63.jpeg


In [13]:
imagePath = '../dataset_validation/covid_validation/6C94A287-C059-46A0-8600-AFB95F4727B7.jpeg'
test_rx_image_for_Covid19_2(covid_pneumo_model, imagePath)

('POSITIVE', 96.0)

### Testing on normal images:

In [14]:
imagePath = '../dataset_validation/normal_validation/IM-0177-0001.jpeg'
test_rx_image_for_Covid19_2(covid_pneumo_model, imagePath)

('NEGATIVE', 99.0)

### Testing With Non Covid - Pneumonia

In [15]:
imagePath = '../dataset_validation/non_covid_pneumonia_validation/person63_bacteria_306.jpeg'
test_rx_image_for_Covid19_2(covid_pneumo_model, imagePath)

('NEGATIVE', 98.0)

# Running a Python Script

In [48]:
ls

10_AI_Xray_Covid19_Detection.ipynb
20_AI_Xray_Covid19_Pneumo_Detection.ipynb
30_AI_Xray_Covid19_Pneumo_Detection-Hyperparameters_test.ipynb
40_AI_Xray_Normal_Pneumo_Detection.ipynb
50_AI_Xray_Covid19_Pneumo_Detection_Application.ipynb
covidXrayApp_test.ipynb
covidXrayApp_test.py
plot.png


In [50]:
! python covidXrayApp_test.py

### 